In [ ]:
from sklearn.datasets import fetch_lfw_people

lfw_dataset = fetch_lfw_people(min_faces_per_person=70, resize=0.4)
lfw_dataset

In [ ]:
# Display a few face images
fig, axarr = plt.subplots(1, 5, figsize=(12, 5))
for i in range(5):
    axarr[i].imshow(lfw_dataset.images[i], cmap='gray')
    axarr[i].set_title(lfw_dataset.target_names[lfw_dataset.target[i]])
    axarr[i].axis('off')
plt.show()